# LangchainNotionToolkit

This notebook provides a quick overview for getting started with **LangchainNotionToolkit**.  
For detailed documentation of all LangchainNotionToolkit features and configurations head to the [API reference](https://pypi.org/project/langchain-notion/).

You may also want to check the official [Notion API docs](https://developers.notion.com/docs) for details on objects like pages, databases, and properties.

---

## Overview

### Integration details

| Class | Package | Serializable | JS support | Package latest |
| :--- | :--- | :---: | :---: | :---: |
| `LangchainNotionToolkit` | [langchain-notion](https://pypi.org/project/langchain-notion/) | N/A | N/A | ![PyPI - Version](https://img.shields.io/pypi/v/langchain-notion?style=flat-square&label=%20) |

### Tool features

The `LangchainNotionToolkit` bundles multiple tools for interacting with Notion through agents:

| Tool | Description | Input Schema |
| :--- | :--- | :--- |
| Search Pages | Search for pages in Notion using keywords. | `{"query": str}` |
| Get Page | Retrieve a page’s title, URL, and properties by ID. | `{"page_id": str}` |
| Create Page | Create a new page in a database or under another page. | `{"parent": {...}, "properties": {...}}` |
| Update Page | Update properties of an existing page. | `{"page_id": str, "properties": {...}}` |

---

## Setup

Install dependencies:

%pip install --quiet -U langchain-community

### Credentials

This integration requires you to set NOTION_API_KEY as an environment variable to authenticate with the Discord API.

`export NOTION_API_KEY="your-notion-token"`

In [ ]:
import getpass
import os

# if not os.environ.get("NOTION_API_KEY"):
#     os.environ["NOTION_API_KEY"] = getpass.getpass("NOTION API key:\n")

It's also helpful (but not needed) to set up [LangSmith](https://smith.langchain.com/) for best-in-class observability:

In [ ]:
# os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

## Use within an agent

Here we show how to instantiate an instance of the LangchainNotionToolkit tool, with

In [19]:
from langchain_notion.toolkits import LangchainNotionToolkit
from langchain_notion.notion_wrapper import NotionWrapper

key = os.environ["NOTION_API_KEY"]
api = NotionWrapper(api_key=key)
toolkit = LangchainNotionToolkit.from_notion_wrapper(api, include_write_tools=True)
tools = toolkit.get_tools()

# search = next(t for t in tools if t.name == "Search Pages")
# print(search.invoke({"tool_input":"test"}))

ModuleNotFoundError: No module named 'langchain_notion'

## Invocation

### Invoke directly with args

Below is a simple example of calling the tool with keyword arguments in a dictionary.

In [ ]:
search_tool = next(t for t in tools if t.name == "Search Pages")
print(search_tool.invoke({"tool_input": "test"}))

### Invoke with ToolCall

We can also invoke the tool with a model-generated ToolCall, in which case a ToolMessage will be returned:

In [ ]:
# This is usually generated by a model, but we'll create a tool call directly for demo purposes.
model_generated_tool_call = {
    "args": {"tool_input": "test"},
    "id": "1",
    "name": search_tool.name,
    "type": "tool_call",
}
search_tool.invoke(model_generated_tool_call)

## Chaining
Below is a complete example showing how to integrate the **LangchainNotionToolkit** tools in an agent with an LLM.  
This uses `create_react_agent` (from `langgraph.prebuilt`) to wire an agent that can call tools when appropriate.

`export NOTION_API_KEY="secret_..."` (and optionally `NOTION_DATABASE_ID`)

Install: `pip install langchain-notion langgraph langchain-openai` (replace LLM as needed)


In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI  # or any LangChain-compatible chat model
from langchain_notion.notion_wrapper import NotionWrapper
from langchain_notion.toolkits import LangchainNotionToolkit
import os

# 1) Instantiate your LLM (replace with your provider/model of choice)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 2) Create the Notion wrapper and toolkit
api = NotionWrapper(api_key=os.environ["NOTION_API_KEY"])
# Set include_write_tools=True if you want Create/Update tools available
toolkit = LangchainNotionToolkit.from_notion_wrapper(api, include_write_tools=True)

# 3) Collect the tools the agent can use
tools = toolkit.get_tools()

# (Optional) Inspect available tools
for t in tools:
    print(f"- {t.name}: {t.description}")

# 4) Build a ReAct-style agent that can call these tools
agent_executor = create_react_agent(llm, tools)

# 5) Formulate a user query that may trigger tool calls.
# Tip: Be explicit about arguments the tools need (e.g., page_id, parent, properties).
example_query = """
Search Notion for pages about 'roadmap'. Then get details for the top result.
If needed, you may call 'Search Pages' and then 'Get Page'.
"""

# 6) Execute the agent (streaming example)
events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)

# 7) Print out the model's responses (and any tool outputs) as they arrive
for event in events:
    event["messages"][-1].pretty_print()


## API reference

For detailed documentation of all LangchainNotionToolkit features and configurations head to the API reference:

* [langchain-notion](https://github.com/rujeetjahagirdar/langchain-notion)